In [1]:
import pandas as pd
import numpy as np



In [2]:
df_template = pd.DataFrame(columns=["driver_id, driver_ref, q1_position, q2_position, q3_position, circuit, constructor_standing, driver_standing, year, constructor, quali_position"])

In [3]:
drivers = pd.read_csv("archive/drivers.csv")
qualifying = pd.read_csv("archive/qualifying.csv")
constructor_results = pd.read_csv("archive/constructor_results.csv")
race_results = pd.read_csv("archive/results.csv")
constructors = pd.read_csv("archive/constructors.csv")

In [4]:
driver_ids_list = [1, 4, 853, 854, 822, 830, 815, 847, 849, 846, 817, 839, 840, 20, 852, 842, 841, 8, 832, 844]
drivers_clean = drivers[drivers.driverId.isin(driver_ids_list)].drop(["number", "forename", "surname", "dob", "nationality", "url", "driverRef"], axis=1).reset_index(drop=True)

drivers_clean

,driverId,code
0,1,HAM
1,4,ALO
2,8,RAI
3,20,VET
4,842,GAS
5,815,PER
6,817,RIC
7,822,BOT
8,830,VER
9,832,SAI


In [5]:
qualifying

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
0,1,18,1,1,22,1,1:26.572,1:25.187,1:26.714
1,2,18,9,2,4,2,1:26.103,1:25.315,1:26.869
2,3,18,5,1,23,3,1:25.664,1:25.452,1:27.079
3,4,18,13,6,2,4,1:25.994,1:25.691,1:27.178
4,5,18,2,2,3,5,1:25.960,1:25.518,1:27.236
...,...,...,...,...,...,...,...,...,...
8769,8810,1055,852,213,22,16,1:18.556,NaN,NaN
8770,8811,1055,8,51,7,17,1:18.917,NaN,NaN
8771,8812,1055,854,210,47,18,1:19.117,NaN,NaN
8772,8813,1055,849,3,6,19,1:19.219,NaN,NaN


In [6]:
qualifying = qualifying.drop(["number", "qualifyId", "position"], axis=1)

In [7]:
test1 = qualifying[["raceId", "driverId", "constructorId"]].copy()


In [8]:
test2 = qualifying.drop(["q2", "q3"], axis=1)
test2 = test2.dropna()
test2["q1_position"] = test2.sort_values(["q1"]).groupby("raceId").cumcount() + 1

In [9]:
test3 = qualifying.drop(["q1", "q3"], axis=1)
test3 = test3.dropna()
test3["q2_position"] = test3.sort_values(["q2"]).groupby("raceId").cumcount() + 1

In [10]:
test4 = qualifying.drop(["q1", "q2"], axis=1)
test4 = test4.dropna()
test4["q3_position"] = test4.sort_values(["q3"]).groupby("raceId").cumcount() + 1

In [11]:
test1 = test1.merge(
    test2,
    on=["raceId", "driverId", "constructorId"],
    how="left"
)

In [12]:
test1 = test1.merge(
    test3,
    on=["raceId", "driverId", "constructorId"],
    how="left"
)

test1 = test1.merge(
    test4,
    on=["raceId", "driverId", "constructorId"],
    how="left"
)

In [13]:
cleaned_positions = test1.drop(["q1", "q2", "q3"], axis=1)

In [14]:
test_x = cleaned_positions.groupby(["raceId"]).transform(lambda x: x.fillna(x.max() + 1))
test_x["raceId"] = cleaned_positions["raceId"]

In [15]:
qualifying = test_x




In [16]:
qualifying

,driverId,constructorId,q1_position,q2_position,q3_position,raceId
0,1,1,10.0,1.0,1.0,18
1,9,2,4.0,2.0,2.0,18
2,5,1,1.0,3.0,3.0,18
3,13,6,3.0,5.0,4.0,18
4,2,2,2.0,4.0,5.0,18
...,...,...,...,...,...,...
8769,852,213,16.0,16.0,11.0,1055
8770,8,51,17.0,16.0,11.0,1055
8771,854,210,18.0,16.0,11.0,1055
8772,849,3,19.0,16.0,11.0,1055
